In [3]:
import torch
from transformers import AutoTokenizer, AutoModel

In [4]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b-int4", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm2-6b-int4", trust_remote_code=True).half().cuda()
model = model.eval()

In [9]:
def prompt_entity_extraction(sentence):
    entity_type = ",".join(["巡查项目","巡查内容","巡查方法","巡查周期"])
    entity_format="[实体1,实体2,实体3,...]"
    return f"\
说明：从给定的输入文本中提取实体，这些实体可能的类型包括“{entity_type}”,提取的实体最后以{entity_format}的格式回答。\n\
示例如三个反引号内所示：\n\
```\n\
输入文本：“管道泄水及冲洗水的封堵应该怎么做？”\n\
输出结果：[管道,泄水及冲洗水,封堵]\n\
```\n\
注意：输出结果是从用户的输入文本中提取实体,输出结果严格以{entity_format}的格式回答,不需要生成其他任何多余的内容。\n\
此时用户的输入文本是：“{sentence}”\n\
输出结果："
sentence = "管廊中渗漏水的维护该怎么做？"
prompt = prompt_entity_extraction(sentence)

In [10]:
print(prompt)

说明：从给定的输入文本中提取实体，这些实体可能的类型包括“巡查项目,巡查内容,巡查方法,巡查周期”,提取的实体最后以[实体1,实体2,实体3,...]的格式回答。
示例如三个反引号内所示：
```
输入文本：“管道泄水及冲洗水的封堵应该怎么做？”
输出结果：[管道,泄水及冲洗水,封堵]
```
注意：输出结果是从用户的输入文本中提取实体,输出结果严格以[实体1,实体2,实体3,...]的格式回答,不需要生成其他任何多余的内容。
此时用户的输入文本是：“管廊中渗漏水的维护该怎么做？”
输出结果：


In [11]:
encoded_input = tokenizer(prompt, return_tensors='pt').to('cuda:0')
output = model(**encoded_input)

In [14]:
probabilities = torch.nn.functional.softmax(output.logits, dim=-1)[:,-1]
next_token_id = torch.argmax(probabilities, dim=-1) 
next_token = tokenizer.decode(next_token_id[0], skip_special_tokens=True)
print(next_token)

["


In [13]:
top_values, top_indices = torch.topk(probabilities.view(-1), 50)
for i in range(0,len(top_indices)):
    print(tokenizer.decode(top_indices[i]),"概率为:",top_values[i].item())

[" 概率为: 0.669921875
[' 概率为: 0.134033203125
[ 概率为: 0.134033203125
[' 概率为: 0.021209716796875
[ 概率为: 0.01551055908203125
[" 概率为: 0.00531768798828125
无法 概率为: 0.005237579345703125
[- 概率为: 0.00417327880859375
〔 概率为: 0.003078460693359375
[\ 概率为: 0.0022869110107421875
[[ 概率为: 0.0018968582153320312
[] 概率为: 0.0003535747528076172
在 概率为: 0.0003399848937988281
无 概率为: 0.0003294944763183594
只能 概率为: 0.00015091896057128906
不需要 概率为: 0.00014972686767578125
管道 概率为: 0.00012803077697753906
<0x0A> 概率为: 0.00010454654693603516
([ 概率为: 9.66787338256836e-05
［ 概率为: 9.22083854675293e-05
管 概率为: 8.20159912109375e-05
[$ 概率为: 6.389617919921875e-05
的唯一 概率为: 5.21540641784668e-05
[] 概率为: 3.904104232788086e-05
没有 概率为: 3.904104232788086e-05
【 概率为: 3.612041473388672e-05
得不到 概率为: 3.3915042877197266e-05
需要在 概率为: 3.0159950256347656e-05
无需 概率为: 2.9027462005615234e-05
=[ 概率为: 2.5391578674316406e-05
可以 概率为: 2.5391578674316406e-05
{" 概率为: 2.5212764739990234e-05
唯一的 概率为: 2.1398067474365234e-05
将 概率为: 2.1398067474365234e-05
[[ 概率为: 